# Extensions to $v$-advection

A v-advection begins with the orchestrator routine lib.split:

In [ ]:
            elif coeff[s] == 'b': # advect v
                Ex = DECSKS.lib.fieldsolvers.Gauss(sim_params['ni'], f, x, vx, n-1, sim_params) # calculate accelerations at time zero (n-1)
                ax = DECSKS.lib.domain.Setup(sim_params, 'a', 'x')
                ax.prepointvaluemesh = -Ex
 
                vx.CFL.compute_numbers(vx, ax, split_coeff*t.width)
                f[n,:,:] = DECSKS.lib.convect.scheme(
                    f[n-1,:,:],
                    n,
                    sim_params,
                    z = vx,
                    vz = ax)

The electric field calculation has been compatibilized in the notebook DECSKS-09 part 4., the instance <code>ax</code> takes the form

$$\text{ax} = \underline{\underline{a}}_x = \left(\begin{array}{cccc}
a_0 & a_0 & \ldots & a_0 \\
a_1 & a_1 & \ldots & a_1 \\
\vdots & \vdots & & \vdots \\
a_{N_x-1} & a_{N_x-1} & & a_{N_x-1}\end{array}\right)$$

i.e. each velocity cell is advected along its row with the computed <code>ax</code>.

The CFL computation precedes the call to the lib.convect.scheme method:

    vx.CFL.compute_numbers(vx, ax, split_coeff*t.width)

Which takes the form:

In [ ]:
    def compute_numbers(self, z, vz, dt):
        """Calculates the CFL numbers and corresponding integer and fractional
        parts for each col of z.prepointmesh and stores in the 2D stack

            z.CFL.compute_numbers(z,vz,dt)

        note that each number corresponds to advection in 1D for each 1D problem
        whose subdomain is the column, and whose column space constitutes the entire
        grid.

        Hence, we implement the indicial displacement of each gridpoint according
        to the velocity values in each column by shifting from prepoints

            (i,j) --> (i,j+ CFL.numbers[j])

        where the index i is not explicitly referenced given it is obvious.n

        inputs:
        self -- (lib.domain.CourantNumber instance) CFL instance with attribute containers
                containers CFL.numbers, CFL.int, CFL.frac.

                NOTE: the method is an attribute belonging to the subinstance z.CFL
                hence, self refers to z.CFL, not z.

        z -- (lib.domain.Setup instance) phasespace instance being advected
        vz -- (lib.domain.Setup instance) velocity for self, e.g. vx, ..., ax, ..
        dt -- (float) width of time step, can be a fraction of t.width for split
              methods

        outputs:
        None -- updates attributes
        """
        self.numbers = vz.prepointvaluemesh * dt / z.width

        # if >= 0 , self.int = floor(self.numbers), else ceil(self.numbers)
        # i.e. the sign of CFL.frac agrees with the sign of vz
        self.int = np.where(self.numbers >=0, np.floor(self.numbers),
                            np.ceil(self.numbers)).astype(int)

        # remaining portion is the fractional CFL number
        self.frac = self.numbers - self.int

        # format dtype as int
        self.int = np.array(self.int, dtype = int)

No change is needed since here, <code>vz = ax</code> and <code>z = x</code>, the appropriate numbers are computed from the calculation:

        self.numbers = vz.prepointvaluemesh * dt / z.width
        
This gives the CFL numbers (and int and frac parts) the same form as <code>ax</code> above.

Next, the function lib.convect.scheme is stepped into:

In [ ]:
def scheme(
    f_initial,
    n,
    sim_params,
    z,
    vz
    ):
    """Solves a collection of 1D advection (in z) equations by convected scheme
    and stacks the results in a 2D matrix

    inputs:
    f_initial -- (ndarray, ndim = 2) f[n-1,:,:] if first
             substep in a splitting algorithm or if none
             else, f[n,:,:]
    n -- (int) time step            
    sim_params -- (dict) simulation parameters
    z -- (instance) phase space variable
    vz -- (instance) generalized velocity for z


    outputs:
    f_final -- (ndarray, ndim = 1 or 2) f[n,:] or f[n,:,:] updated
               after all steps have been completed
    """

    # (0) INITIALIZE FINAL DENSITY CONTAINER AND EXTRACT EVOLVED GRID
    f_final = np.zeros(f_initial.shape)
    f_initial = DECSKS.lib.convect.extract_active_grid(sim_params, f_initial)

    # (1) PUSH MOVING CELLS an integer number of cells
    z.postpointmesh = advection_step(z)

    # (*) APPLY BOUNDARY CONDITIONS
    z.postpointmesh = DECSKS.lib.boundaryconditions.periodic(z.postpointmesh, z.N)

    # (2) REMAP DENSITY TO GRID AND APPLY BCs IF NEEDED
    f_remapped = remap_step(
                       sim_params,
                       f_initial,
                       n,
                       z,
                       vz
                       )

    # (3) COLLISION STEP (NOT YET IMPLEMENTED)
    # f_new = DECSKS.lib.collisions.collisiontype(f_old, z, n)

    # (4) RETURN FINAL DESTINY (density*)
    f_final = finalize_density(sim_params, f_remapped, f_final)


Stepping through key lines:

    L28: f_initial = DECSKS.lib.convect.extract_active_grid(sim_params, f_initial)
    
extracts a density function with shape f.shape = (x.N, v.N)

    L31: z.postpointmesh = advection_step(z)
    
calls the lib.convect.advection_step method:

In [ ]:
def advection_step(z):
    """Pushes each z.prepointmesh (index) value by the advection *along each
    column j* (i.e. constant vz.prepointvaluemesh)
    as prescribed by its generalized velocity vz.prepointmeshvalues[:,j].

    This is computed in one calculation by:

        vz.prepointmeshvalues * dt / z.width = CFL.numbers

    the integer part, CFL.int, is pushed here. The residual fraction
    CFL.frac is remapped according to remap_step(*args) in step (2) in
    the orchestrator routine, scheme(*args), above.

    inputs:
    z -- (instance) phase space variable equipped with (among other attributes)

            z.prepointmesh -- (ndarray, ndim=2) initial [i,j] prepoint indices of all MCs

            z.CFL -- (instance) contains CFL numbers, int and frac parts
                z.CFL.numbers -- (ndarray, ndim=2, dtype=float64)
                z.CFL.int -- (ndarray, ndim=2, dtype=int) integer part of CFL numbers
                z.CFL.frac -- (ndarray, ndim=2, dtype=float64) fractional part

    outputs:
    z -- (instance) phase space variable updated with postpointmesh attribute (BCs not applied)

        updated attr:
        z.postpointmesh -- (ndarray, ndim=3, dtype=int), shape = (2, x.N, v.N) matrix containing each pair
                           k[:,i,j] of postpoints for each prepoint [i,j]


    NOTE: Boundary conditions NOT applied at this point

    USAGE NOTE: the map z.postpointmesh[k,i,j] for k = 0 or 1 gives the postpoint
      of the advecting index, it does not give the relevant postpoint duple
      in phase space:

          (z.postpointmesh[k,i,j], j) [for x advection]

      or

          (i, z.postpointmesh[k,i,j]) [for v advection]

      but instead just gives that value z.postpointmesh[k,i,j] of the index
      that changes. Hence, the remap procedure must be encoded with an
      understanding of which index is updated in order to complete the push.

      It then makes sense to speak in terms such as:

          z.postpointmesh[0,i,j] + 1 is a contiguous gridpoint to
                       the index z.postpointmesh[0,i,j] in the positive
                       direction

          z.postpointmesh[0,i,j] - 1 is a contiguous gridpoint to
                       the index z.postpointmesh[0,i,j] in the negative
                       direction

      since we are speaking of one index, not a duple as concerns the postpointmesh
      object
    """
    z.postpointmesh[0,:,:] = z.prepointmesh + z.CFL.int
    z.postpointmesh[1,:,:] = np.sign(z.CFL.frac).astype(int) + z.postpointmesh[0,:,:]

    return z.postpointmesh

This also is computed consistently with vx as the advecting variable. Next,

    L36: z.postpointmesh = DECSKS.lib.boundaryconditions.periodic(z.postpointmesh, z.N)
    
the method has the following implementation:

In [ ]:
def periodic(w, Nw):
    """Applies periodic boundary conditions to
    postpoints

    inputs:
    w -- (ndarray, ndim=arbitrary) array whose indices are to be
         kept in range 0, 1, ... , S-1 per periodic BCs
    S -- (int) supremum value in the modular arithematic operation
         for example, S = z.N restricts indices between 0, 1, ... , z.N

    outputs:
    Array with periodic BCs enforced according to bound on attribute4 S.N
    """

    return np.mod(w, Nw)


This is also computed consistently. Finally, we have the remap step of the algorithm:

    # (2) REMAP DENSITY TO GRID
    f_remapped = remap_step(
                       sim_params,
                       f_initial,
                       n,
                       z,
                       vz
                       )

The remap_step is a top-level conductor for calling many methods that are used to compute the high order fluxes and eventually to remap the density packets to contiguous gridpoints appropriately.

In [ ]:
def remap_step(
        sim_params,
        f_old,
        n,
        z,
        vz
        ):
    """Orchestrates remapping of all advected moving cells to the grid,
    the flowchart of methods being used is the following:

    f_final = convect.remap_step(
               sim_params,
                f_old,
                n,
                z,
                vz
                )

                | |
               \   /
                \ /

    Uf = convect.flux(    ->   c = HOC.correctors(sim_params, z, vz)
        sim_params,
        f_old,           ->   d = derivatives.method(f, z, vz, sim_params)
        z, vz
        )                <-   Uf = sum over q (c[q,:]*d[q,:,:])


                | |
               \   /
                \ /


   remap the appropriate proportion to the nearest neighbor gridpoints
   f_k1 = convect.remap_assignment(
                            f_old,
                            Uf,
                            z.postpointmesh[0,:,:],
                            z,
                            vz,
                            index = 'nearest'
                            )


   remap the remaining proportion to the appropriate contiguous gridpoint
   f_k2 = convect.remap_assignment(
                            f_old,
                            Uf,
                            z.postpointmesh[1,:,:],
                            z,
                            vz,
                            index = 'contiguous'
                            )

   return f_remapped = f_k1 + f_k2

                | |
               \   /
                \ /


   f_final = convect.finalize_density(
                            sim_params,
                            f_remapped,
                            f_final # initialized container of zeros
                            )

    inputs:
    sim_params -- (dict) simulation parameters
    f_old -- (ndarray, dim=2) density from previous time step, full grid
    n  -- (int) current time step
    z -- (instance) phase space variable
    vz -- (instance) generalized velocity variable for phase space variable z

    outputs:
    f_remapped -- (ndarray, dim=2) density with all MCs remapped at final postpoints
                     according to remap rule

    NOTE: we carry assemble the new density as a sum of two
    2D matrices f_k1 and f_k2 which themselves are the associated
    mappings to the postpoints k1 (nearest neighbor grid index to
    exact non-integral postpoint), and k2 (contiguous grid index
    to k1 based on the sign of advection), we use k here for brevity
    but the postpoint pairs are stored in z.postpointmesh[:,i,j] for
    each [i,j]

    The reason a partition is necessary is that a mapping such as

        f_new[k, vz.prepointmesh] += f_old

    does not increment each element, but instead overwrites leaving
    only the final assignment to a particular gridpoint rather than
    adds on top of it. Thus, the separation of k1 and k2 into
    different containers f_k1 and f_k2 permit non-overlapping
    assignments in each individually since every z is pushed by the same
    vz for a given row (column), so overlapping assignments are not
    possible for each index separately. The sum gives the appropriate ]
    total density f_new.
    """

    # compute high order fluxes
    Uf = flux(
        sim_params,
        f_old,
        z, vz
        )

    # remap to nearest neighbor cell center
    f_k1 = remap_assignment(
        f_old,
        Uf,
        z.postpointmesh[0,:,:],
        z,
        vz,
        index = 'nearest'    # remaps to nearest neighbor index
        )

    # remap to contiguous cell center
    f_k2 = remap_assignment(
        f_old,
        Uf,
        z.postpointmesh[1,:,:],
        z,
        vz,
        index = 'contiguous'    # remaps to contiguous neighbor of above
        )

    f_remapped = f_k1 + f_k2

    # global check on L1 norm
    DECSKS.lib.density.global_conservation_check(sim_params, f_remapped, n)

    return f_remapped

At line 103 of convect.remap_step, the flux function is called:

             # compute high order fluxes
    L103:    Uf = flux(
                sim_params,
                f_old,
                z, vz
                )
which has the following implementation:

In [ ]:
def flux(
        sim_params,
        f_old,
        z, vz
        ):
    """Computes fluxes Uf for all z.prepointmesh

    inputs:
    f_old -- (ndarray, dim=2) density from previous time (sub)step
    CFL -- (instance) CFL number
    z -- (instance) phase space variable
    sim_params -- (dict) simulation parameters

    outputs:
    Uf -- (ndarray, dim=2) Normalized fluxes for every z.prepoints[i]

           where for each [i,j]:

               Uf[i,j] = sum c[q,j]*d[q,i,j] over q = 0, 1, ... N-1
    """

    c = DECSKS.lib.HOC.correctors(sim_params, z, vz)

    # evaluate derivatives q = 0, 1, 2, ... N-1 (zeroeth is density itself)

    # calls lib.derivatives.fd or lib.derivatives.fourier based on the
    # HOC specified in etc/params.dat, sim_params['derivative_method']
    # contains the related function handle as a string
    d = eval(sim_params['derivative_method'])(f_old, z, vz, sim_params)

    # compute high order fluxes column-by-column
    Uf = np.zeros(f_old.shape)
    for j in range(vz.N):
        Uf[:,j] = c[:,j].dot(d[:,:,j])

    # enforce flux limiter to ensure positivity and restrict numerical overflow
    Uf = flux_limiter(f_old, Uf, z)

    return Uf


We have

    L22:     c = DECSKS.lib.HOC.correctors(sim_params, z, vz)

where,

In [ ]:
def correctors(sim_params, z, vz):
    """computes the correction coefficients c for every [i,j]

    inputs:
    sim_params -- (dict) simulation parameters

    outputs:
    c -- (ndarray, ndim=2) correction coefficients
         with shape = (N, z_notadv.N)
         where z_notadv means the not advecting phase
         space variable in the scope of a 2D advection
         implementation

         per DECSKS-09 notation, the tensor c is given by

             c = I_alternating.dot(B)

        where I_alternating.shape = (N,N), and the entries

             I_alternating[i,i] = (-1) ** i
             I_alternating[i,j] = 0 for all i != j

        and the matrix B.shape = (N,vz.N) is the vectors
        of beta correctors (shape = (N,1)) for each value
        of vz.prepointvaluemesh[:,j]
    """
    Beta_func_handle = "%s%s" % ("Beta_matrix_", z.str[0])

    B = eval(Beta_func_handle)(sim_params, z, vz)
    c = sim_params['I_alternating'].dot(B)
    return c


The computation of <code>c</code> is accomplished as (above written for x-advection). <b>For x-advection</b> we had:

$$\underline{\underline{c}}_{x,N_x\times N_v} = \underline{\underline{I}}^{\pm}\cdot \underline{\underline{B}}_x$$


<b>Recall</b>, for the case of $x$ moving cells in advected, we have $\alpha = \alpha_j$, thus $\beta^{(i,j)}_p = \beta^{,j}_p$ for all $i$ and we save $N_x$ computations by constancy in $i$ at each $j$ (as well as storage). For every $j$ ($v_j$), we can find the associated $N$ correctors $\{\beta^{,j}\}$ that corresponds to all $x_i$ at that velocity as the solution to:

$$\left(\begin{array}{c}
\beta^{,j}_0 \\
\beta^{,j}_1 \\
\beta^{,j}_2 \\
\beta^{,j}_3 \\
\vdots \\
\vdots \\
\beta^{,j}_{N-3} \\
\beta^{,j}_{N-2} \\
\beta^{,j}_{N-1} 
\end{array}\right)_{N\times 1} = \left(\begin{array}{ccc c ccc}
\phantom{\pm}\frac{B_0}{0!} & \phantom{\pm}0 & \cdots & \cdots & \cdots & \cdots & \cdots & \cdots & 0\\
\pm \frac{B_1}{1!} & \phantom{\pm}\frac{B_0}{0!} & 0 & \cdots & \cdots & \cdots & \cdots & \cdots & 0 \\
\phantom{\pm}\frac{B_2}{2!} & \pm \frac{B_1}{1!} & \phantom{\pm}\frac{B_0}{0!} & 0 & \cdots &  \cdots & \cdots &  \cdots & 0 \\
 \phantom{\pm}\frac{B_3}{3!}& \phantom{\pm}\frac{B_2}{2!} & \pm \frac{B_1}{1!} & \frac{B_0}{0!} & 0 &  \cdots & \cdots  &  \cdots &0 \\
\phantom{\pm}\vdots & \phantom{\pm}\vdots & \ddots & \ddots & \ddots & \ddots &  & & 0\\
\phantom{\pm}\vdots & \phantom{\pm}\vdots &  & \ddots & \ddots & \ddots & \ddots & & 0 \\
\phantom{\pm}\frac{B_{N-3}}{(N-3)!} & \phantom{\pm}\frac{B_{N-4}}{(N-4)!} & \cdots & \cdots & \ddots & \pm \frac{B_1}{1!} & \phantom{\pm}\frac{B_0}{0!} & 0 & 0\\
\phantom{\pm}\frac{B_{N-2}}{(N-2)!} & \phantom{\pm}\frac{B_{N-3}}{(N-3)!} & \cdots  & \cdots & \cdots & \phantom{\pm}\frac{B_2}{2!} & \pm \frac{B_1}{1!} & \frac{B_0}{0!} & 0\\
\phantom{\pm}\frac{B_{N-1}}{(N-1)!} & \phantom{\pm}\frac{B_{N-2}}{(N-2)!} & \cdots & \cdots & \cdots & \phantom{\pm}\frac{B_3}{3!} & \phantom{\pm}\frac{B_2}{2!} & \pm \frac{B_1}{1!} & \frac{B_0}{0!}\\
\end{array}\right)_{N\times N} \left(\begin{array}{c}
\frac{\alpha_{,j}^1}{1!} \\
\frac{\alpha_{,j}^2}{2!} \\
\frac{\alpha_{,j}^3}{3!}\\
\vdots \\
\vdots \\
\frac{\alpha_{,j}^{N-2}}{(N-2)!}\\
\frac{\alpha_{,j}^{N-1}}{(N-1)!} \\
\frac{\alpha_{,j}^N}{N!}
\end{array}
\right)_{N\times 1}, \qquad \text{x advection} \qquad \text{for all } j\in [0,1,\ldots , N_v - 1]$$


For $v$-advection, we have $\alpha = \alpha (x_i)$ (correspondingly $\beta = \beta (x_i)$), so the according changeovers required are $\alpha_{,j} \rightarrow \alpha_i$, where the entries are constant for each column entry in a given row (before we had each row being constant for each column). There is not a way to get around a transpose, either we commute the dot product ordering so that in the v-advection case we have $\underline{\underline{\tilde{\alpha}}}_v\cdot \underline{\underline{A}}^T = \underline{\underline{B}}_v$, or we transpose the alpha matrix $\underline{\underline{\alpha}}_v$.

A test of the transpose operation from numpy shows the following sort of time expense

In [1]:
import numpy as np
A = np.random.randn(768, 1536)

In [2]:
%%timeit

np.transpose(A)

The slowest run took 30.03 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 397 ns per loop


So even for large grids, this operation is not of substantial cost.

To visualize the transposition, first recall the $\alpha$ tensor for $v$-advection takes the form:

$$\underline{\underline{\alpha}}_v = (\alpha_i)_v = \left(\begin{array}{ccccc}
\alpha_{0} & \alpha_{0} & \cdots & \alpha_{0} & \alpha_{0} \\
\alpha_{1} & \alpha_{1} & \cdots & \alpha_{1} & \alpha_{1} \\
\vdots & \vdots && \vdots & \vdots \\
\alpha_{N_x-2} & \alpha_{N_x-2} & \cdots & \alpha_{N_x-2} & \alpha_{N_x-2} \\
\alpha_{N_x-1} & \alpha_{N_x-1} & \cdots & \alpha_{N_x-1} & \alpha_{N_x-1} \\
\end{array}
\right)_{N_x\times N_v} \qquad \text{for } v \text{ advection}$$

The transpose ($\underline{\underline{\alpha}}^T = (\alpha^T_v)_{i,j} = (\alpha_v)_{j,i}$) we recover a similar looking result to the case of $x$-advection, where we 
drop the signature $v$ for brevity in notation, 

$$\underline{\underline{\alpha}}^T = (\alpha_{,j}^T) = \left(\begin{array}{ccccc}
\alpha_{,0} & \alpha_{,1} & \cdots & \alpha_{,N_x- 2} & \alpha_{,N_x-1} \\
\alpha_{,0} & \alpha_{,1} & \cdots & \alpha_{,N_x- 2} & \alpha_{,N_x-1} \\
\vdots & \vdots & & \vdots & \vdots \\
\alpha_{,0} & \alpha_{,1} & \cdots & \alpha_{,N_x- 2} & \alpha_{,N_x-1} \\
\alpha_{,0} & \alpha_{,1} & \cdots & \alpha_{,N_x- 2} & \alpha_{,N_x-1} \\
\end{array}
\right)_{N_v \times N_x} \qquad \text{for } v \text{ advection}$$

Thus, each $\alpha_i$ which is constant in each row is transferred to being constant in each column $j$ and represented as $\alpha^T_{,j}$ just as for x-advection.

$$\left(\begin{array}{c}
\beta^{i}_0 \\
\beta^{i}_1 \\
\beta^{i}_2 \\
\beta^{i}_3 \\
\vdots \\
\vdots \\
\beta^{i}_{N-3} \\
\beta^{i}_{N-2} \\
\beta^{i}_{N-1} 
\end{array}\right)_{N\times 1} = \left(\begin{array}{ccc c ccc}
\phantom{\pm}\frac{B_0}{0!} & \phantom{\pm}0 & \cdots & \cdots & \cdots & \cdots & \cdots & \cdots & 0\\
\pm \frac{B_1}{1!} & \phantom{\pm}\frac{B_0}{0!} & 0 & \cdots & \cdots & \cdots & \cdots & \cdots & 0 \\
\phantom{\pm}\frac{B_2}{2!} & \pm \frac{B_1}{1!} & \phantom{\pm}\frac{B_0}{0!} & 0 & \cdots &  \cdots & \cdots &  \cdots & 0 \\
 \phantom{\pm}\frac{B_3}{3!}& \phantom{\pm}\frac{B_2}{2!} & \pm \frac{B_1}{1!} & \frac{B_0}{0!} & 0 &  \cdots & \cdots  &  \cdots &0 \\
\phantom{\pm}\vdots & \phantom{\pm}\vdots & \ddots & \ddots & \ddots & \ddots &  & & 0\\
\phantom{\pm}\vdots & \phantom{\pm}\vdots &  & \ddots & \ddots & \ddots & \ddots & & 0 \\
\phantom{\pm}\frac{B_{N-3}}{(N-3)!} & \phantom{\pm}\frac{B_{N-4}}{(N-4)!} & \cdots & \cdots & \ddots & \pm \frac{B_1}{1!} & \phantom{\pm}\frac{B_0}{0!} & 0 & 0\\
\phantom{\pm}\frac{B_{N-2}}{(N-2)!} & \phantom{\pm}\frac{B_{N-3}}{(N-3)!} & \cdots  & \cdots & \cdots & \phantom{\pm}\frac{B_2}{2!} & \pm \frac{B_1}{1!} & \frac{B_0}{0!} & 0\\
\phantom{\pm}\frac{B_{N-1}}{(N-1)!} & \phantom{\pm}\frac{B_{N-2}}{(N-2)!} & \cdots & \cdots & \cdots & \phantom{\pm}\frac{B_3}{3!} & \phantom{\pm}\frac{B_2}{2!} & \pm \frac{B_1}{1!} & \frac{B_0}{0!}\\
\end{array}\right)_{N\times N} \left(\begin{array}{c}
\frac{(\alpha^T_{,j})^1}{1!} \\
\frac{(\alpha^T_{,j})^2}{2!} \\
\frac{(\alpha^T_{,j})^3}{3!}\\
\vdots \\
\vdots \\
\frac{(\alpha^T_{,j})^{N-2}}{(N-2)!}\\
\frac{(\alpha^T_{,j})^{N-1}}{(N-1)!} \\
\frac{(\alpha^T_{,j})^N}{N!}
\end{array}
\right)_{N\times 1}, \qquad \text{v advection} \qquad \text{for all } i\in [0,1,\ldots , N_x - 1]$$

This reproduces the model used in $x$-advection, so we can then assemble the $\tilde{\alpha}$ tensor as

To compute the scaled vector $\underline{\tilde{\alpha}}_{i} = (\underline{\tilde{\alpha}}_{i})_{N\times 1}$, we use numpy vectorized computations:

    import scipy
    import numpy as np
    
    alpha_tilde_i = CFL.frac[:,0] ** np.arange(1,N+1) / scipy.misc.factorial(np.arange(1,N+1))
    
<b>we can take any j = 0, 1, ..., vx.N as every row has constant CFL.frac $\equiv \alpha$. Hence all rows are identical</b> (cf. above matrix for $\underline{\underline{\alpha}}$), here arbitrarily take i = 0. In pseudo-math/code, we are computing:

$$(\underline{\tilde{\alpha}_{i}})_{N\times 1} = \frac{\alpha_j^{(1, 2, \ldots , N)}}{(1, 2, \ldots , N)!}, \qquad i\in [0,1,\ldots , N_x - 1]$$

Which computes by efficient vectorized computation, the following in proper mathematics:

$$\tilde{\alpha}_{i} = \frac{\alpha_i^p}{p!} \quad \text{where } p\in [1,2,\ldots , N]$$

for some $i$. We can do this for all i, then transpose. Alternatively, we can be do this as before with the whole tensor $\underline{\underline{\alpha}}$. We can decide now that we will take a transpose at this point.

    alpha = np.transpose(alpha)    # alpha.shape = (v.N, x.N) now 
    
i.e.

$$\underline{\underline{\alpha}} \rightarrow \underline{\underline{\alpha}}^T$$

where we reuse the same label in the code without mention of it being a transpose, this is normal practice.

We then slice indices 0, 1, 2, ..., N - 1, in the x-advection development this was labelled with a hat diacritic:

$$\underline{\underline{\hat{\alpha}}}^T = \underline{\underline{\alpha}}^T_{N\times N_x} = \text{alpha[:N,:]}$$

this is permissible since there is no loss of information: every row of the transpose is identical. Note, what we need is to raise each row by a power p and divide by the factorial of that same p, where p is the row number for the transpose (same as for x-advection).

We generate the index matrix $\underline{\underline{N}}$ using the ndarray $\underline{N}_{N\times 1} = (1, \ldots , N)^T$. Here we have $N_x$ columns, hence we assemble:

$$\underline{\underline{N}}_{N\times N_x} =  \underline{N}_{N\times 1} \otimes \underline{1}_{1\times N_x} = \left(\begin{array}{ccccc}
1 & 1 & \cdots & 1 & 1 \\
2 & 2 & \cdots & 2 & 2 \\
\vdots & \vdots & & \vdots & \vdots \\
N-1 & N-1 & \cdots & N-1 & N-1 \\
N & N & \cdots & N & N \\
\end{array}
\right)_{N\times N_x}$$





    

We then also define just as before:

$$(\underline{\underline{N}}!)_{N\times N_x} = \left(\begin{array}{ccccc}
1! & 1! & \cdots & 1! & 1! \\
2! & 2! & \cdots & 2! & 2! \\
\vdots & \vdots & & \vdots & \vdots \\
(N-1)! & (N-1)! & \cdots & (N-1)! & (N-1)! \\
N! & N! & \cdots & N! & N! \\
\end{array}
\right)$$

So that the overall matrix $\underline{\underline{\tilde{\alpha}}}_v^T$ can be generated as before:

$$\underline{\underline{\tilde{\alpha}}}_v^T = \frac{(\underline{\underline{\hat{\alpha}}}^T)^{\underline{\underline{N}}}}{\underline{\underline{N}}!}, \qquad \qquad \text{where } \underline{\underline{\hat{\alpha}}} = \underline{\underline{\alpha}}_{N\times N_v} = (\alpha_{i,j})_{i\in [0,1,\ldots N-1], j\in [0,1,\ldots , N_v-1]}$$


Again, the masked implementation will be the most efficient.

### Masked array computation of $\underline{\underline{B}}_v = (\underline{\beta}^{i})_{i=0}^{N_x-1}$, for $\underline{\beta} = \underline{\beta}_{N\times 1}$

$$\underline{\underline{B}}_{v,\text{positive columns}} \equiv \underline{\underline{B}}^+_v = \underline{\underline{A}}^+\cdot \underline{\underline{\tilde{\alpha}}}^T_{\text{masked}}$$

$$\underline{\underline{B}}_{v,\text{negative columns}}\equiv \underline{\underline{B}}^-_v = \underline{\underline{A}}^-\cdot \underline{\underline{\tilde{\alpha}}}^T_{(\neg\text{masked})}$$

where 

$$\underline{\underline{\tilde{\alpha}}}^T_{\text{masked}} = (\alpha_{i,j}) \, \colon \, \alpha_{i,j} \geq 0$$

$$\underline{\underline{\tilde{\alpha}}}^T_{\neg\text{masked}} = (\alpha_{i,j}) \, \colon \, \alpha_{i,j} < 0$$

and

$$
\underline{\underline{B}}_v = (B_{ij})_v = \left\{\begin{array}{ll}
            (B_v)^+_{ij} & \quad \alpha_{ij} \geq 0 \\
            & \\
            (B_v)^-_{ij} & \quad \text{else}
        \end{array}
    \right.
$$


where we have just computed:


$$\underline{\underline{B}}_v = \left[(\underline{\beta}_0)_{N\times 1}, \phantom{a} (\underline{\beta}_1)_{N\times 1}, \phantom{a} \ldots \phantom{a} , \phantom{a} (\underline{\beta}_{N_x-1})_{N\times 1}\right]_{N\times N_x}$$

It then follows just as before that since $c_q = (-1)^q\beta_q(\alpha )$, we have:


$$\underline{\underline{I}}^{\pm}_{N\times N}\cdot\underline{\underline{B}}_v  = \left(\begin{array}{cccccccc}
(-1)^0 & 0 & \cdots & \cdots & \cdots & \cdots  & 0 \\
0 & (-1)^1 & 0  & \cdots & \cdots & \cdots & 0 \\
\vdots & 0 & (-1)^2 & 0 & \cdots & \cdots &  0 \\
\vdots & \vdots & 0 & (-1)^3 & 0 & \cdots &  0 \\
\vdots & \vdots & \vdots & \ddots & \ddots & \ddots & \vdots \\
\vdots & \vdots & \vdots & \vdots & 0 & (-1)^{N-2} & 0\\
\vdots & \vdots & \vdots & \vdots & \vdots & 0 & (-1)^{N-1}\\
\end{array}\right)\cdot\left(\begin{array}{cccccccc}
\beta_0^{,0} & \beta_0^{,1} & \cdots & \cdots & \cdots & \beta_0^{,N_v-2}  & \beta_0^{,N_v-1} \\
\beta_1^{,0} & \beta_1^{,1} & \cdots & \cdots & \cdots & \beta_1^{,N_v-2} & \beta_1^{,N_v-1} \\
\beta_2^{,0} & \beta_2^{,1} & \cdots & \cdots & \cdots & \beta_2^{,N_v-2} &  \beta_2^{,N_v-1} \\
\vdots  & \vdots & &  &  & \vdots & \vdots \\
\vdots & \vdots & &  &  & \vdots & \vdots \\
\beta_{N-2}^{,0} & \beta_{N-2}^{,1} & \cdots & \cdots & \cdots & \beta_{N-2}^{,N_v-2} & \beta_{N-2}^{,N_v -1}\\
\beta_{N-1}^{,0} & \beta_{N-1}^{,1} & \cdots & \cdots & \cdots & \beta_{N-1}^{,N_v-2} & \beta_{N-1}^{,N_v-1} \\
\end{array}\right) =  \left(\begin{array}{cccccccc}
(-1)^0\beta_0^{,0} & (-1)^0\beta_0^{,1} & \cdots & \cdots & \cdots & (-1)^0\beta_0^{,N_v-2}  & (-1)^0\beta_0^{,N_v-1} \\
(-1)^1\beta_1^{,0} & (-1)^1\beta_1^{,1} & \cdots & \cdots & \cdots & (-1)^1\beta_1^{,N_v-2} & (-1)^1\beta_1^{,N_v-1} \\
(-1)^2\beta_2^{,0} & (-1)^2\beta_2^{,1} & \cdots & \cdots & \cdots & (-1)^2\beta_2^{,N_v-2} & (-1)^2\beta_2^{,N_v-1} \\
\vdots  & \vdots & &  &  & \vdots & \vdots \\
\vdots & \vdots & &  &  & \vdots & \vdots \\
(-1)^{N-2}\beta_{N-2}^{,0} & (-1)^{N-2}\beta_{N-2}^{,1} & \cdots & \cdots & \cdots & (-1)^{N-2}\beta_{N-2}^{,N_v-2} & (-1)^{N-2}\beta_{N-2}^{,N_v -1}\\
(-1)^{N-1}\beta_{N-1}^{,0} & (-1)^{N-1}\beta_{N-1}^{,1} & \cdots & \cdots & \cdots & (-1)^{N-1}\beta_{N-1}^{,N_v-2} & (-1)^{N-1}\beta_{N-1}^{,N_v-1} \\
\end{array}\right)_{N\times N_x} = \underline{\underline{c}}_v
$$

## Summary of $\underline{\underline{c}}$ calculations

Thus, we have

#### x-advection:

$$\underline{\underline{c}}_x = \underline{\underline{I}}_{N\times N}^{\pm}\cdot \underline{\underline{B}}_x$$

where
<table style = "width:75%"><tr><td>
$$
\underline{\underline{B}}_x = (B_{ij})_x = \left\{\begin{array}{ll}
            \left(\underline{\underline{A}}^+\cdot \underline{\underline{\tilde{\alpha_x}}}\right)_{ij} & \quad (\alpha_x)_{ij} \geq 0 \\
            & \\
            \left(\underline{\underline{A}}^-\cdot \underline{\underline{\tilde{\alpha_x}}}\right)_{ij} & \quad \text{else}
        \end{array}
    \right.
$$

$$\underline{\underline{\tilde{\alpha}}}_x = \frac{(\underline{\underline{\hat{\alpha_x}}})^{\underline{\underline{N}}}}{\underline{\underline{N}}!}, \qquad \qquad \text{where } \underline{\underline{\hat{\alpha}}} = \underline{\underline{\alpha_x}}_{N\times N_v} = (\alpha_{i,j})_{i\in [0,1,\ldots N-1], j\in [0,1,\ldots , N_v-1]}$$
</td></tr></table>

#### v-advection:

$$\underline{\underline{c}}_v = \underline{\underline{I}}_{N\times N}^{\pm}\cdot \underline{\underline{B}}_v$$

<table style = "width:75%">
<tr><td>
$$
\underline{\underline{B}}_v = (B_{ij})_v = \left\{\begin{array}{ll}
            \left(\underline{\underline{A}}^+\cdot \underline{\underline{\tilde{\alpha_v}}}^T\right)_{ij} & \quad (\alpha^T_v)_{ij} \geq 0 \\
            & \\
            \left(\underline{\underline{A}}^-\cdot \underline{\underline{\tilde{\alpha_v}}}^T\right)_{ij} & \quad \text{else}
        \end{array}
    \right.
$$

$$\underline{\underline{\tilde{\alpha}}}_v^T = \frac{(\underline{\underline{\hat{\alpha}}}^T)^{\underline{\underline{N}}}}{\underline{\underline{N}}!}, \qquad \qquad \text{where } \underline{\underline{\hat{\alpha}}} = \underline{\underline{\alpha}}_{N\times N_v} = (\alpha_{i,j})_{i\in [0,1,\ldots N-1], j\in [0,1,\ldots , N_v-1]}$$
</td></tr></table>

#### Common matrices

$$\underline{\underline{I}}^{\pm}_{N\times N} = \left(\begin{array}{cccccccc}
1 & 0 & \cdots & \cdots & \cdots & \cdots  & 0 \\
0 & -1 & 0  & \cdots & \cdots & \cdots & 0 \\
\vdots & 0 & 1 & 0 & \cdots & \cdots &  0 \\
\vdots & \vdots & 0 & -1 & 0 & \cdots &  0 \\
\vdots & \vdots & \vdots & \ddots & \ddots & \ddots & \vdots \\
\vdots & \vdots & \vdots & \vdots & 0 & (-1)^{N-2} & 0\\
\vdots & \vdots & \vdots & \vdots & \vdots & 0 & (-1)^{N-1}\\
\end{array}\right)$$

$$\underline{N}_{N\times 1} = (1, 2, \ldots , N)^T, \qquad \text{N.B. not }(0,N-1]$$ 

$$\underline{\underline{N}}_{N\times N_x} =  \underline{N}_{N\times 1} \otimes \underline{1}_{1\times N_x} = \left(\begin{array}{ccccc}
1 & 1 & \cdots & 1 & 1 \\
2 & 2 & \cdots & 2 & 2 \\
\vdots & \vdots & & \vdots & \vdots \\
N-1 & N-1 & \cdots & N-1 & N-1 \\
N & N & \cdots & N & N \\
\end{array}
\right)_{N\times N_x}$$


Whether or not we should transpose this back to be of shape (x.N, vx.N) will be determined by the natural shape that d takes given the implementation, we examine this below.

## Assembling $\underline{\underline{d}}_{N\times N_x\times N_v}$ for $v$-advection

### Spectral derivatives

Formerly, we had the function:

### <font color = "magenta">lib.derivatives.fourier</font> (general)

In [ ]:
def fourier(f_old, z, vz, sim_params):
    """Orchestrates the computation of sim_params['N'] - 1 fourier
    derivatives and stores these in a matrix of derivative
    coefficients d. The zeroeth entry contains the zereoth
    derivative, i.e. the density itself

    inputs:
    f_old -- (ndarray, ndim=2) density from previous time substep
    z -- (instance) phase space variable being advected, here
         we are taking d/dz

    vz -- (instance) generalized velocity for z
    sim_params -- (dict) simulation parameters dictionary

    outputs:
    d -- (ndarray, ndim=3), shape = (N, x.N, v.N), the
         entry d[q,i,j] gives the qth derivative coefficient
         at a point [i,j]
    """
    d = np.zeros([sim_params['N'], z.N, vz.N])

    # zeroeth derivative coefficient is the density itself
    d[0,:,:] = f_old

    # if corrections indicated in etc/params.dat, need higher order terms
    if sim_params['N'] > 1:
        d[1:,:,:] = trigonometric3D(f_old, z, sim_params)

    return d

The initialization:

    d = np.zeros([sim_params['N'], z.N, vz.N])

creates a tensor d.shape = (N, x.N, vx.N) for x-advection (z = x, vz = vx), and for v-advection (z = vx, vz = ax, note ax.N = x.N) it creates a tensor of shape d.shape = (N, vx.N, x.N)

The next line presents a potential conflict:

    d[0,:,:] = f_old
    
if f_old has not been transposed, then we have dimensions not aligned given for v-advection, f_old.shape = (x.N, vx.N),k and d[0,:,:].shape = (vx.N, x.N) from the above. <b>if the passed density f_old has the transposed shape, we have no consequence here.</b>

i.e. if early on, in lib.convect.scheme, we modify it to have the transposition.

there is some inconsistency either way, either we transpose f_old early on (and also transpose z.prepointmesh, z.postpointmesh), or we make a special function for v advection and change the one line above so that

    d = np.zeros([sim_params['N'], vz.N, z.N])
    
Then the above statement d[0,:,:] = f_old is consistent, but then transforming along the axes in trigonometric3D below is reversed for v-advection. This is necessarily becoming entangled, we provide the x-advection function below for reference, but start from the ground up thereafter to develop the appropriate object handling:

### <font color = "magenta">lib.derivatives.trigonometric3D</font> (x advection)

In [ ]:
def trigonometric3D(f, z, sim_params):
    """Computes derivatives of density for derivative coeffs d in FN methods

    inputs:
    f -- (ndarray, ndim=2) density at a given time step
    sim_params['Xi']['x' or 'vx'] -- (ndarray, ndim=3) Xi = (1j*dz*xi) ** [[[1,2,3,...,N-1]]], where xi.shape (x.N, v.N)
          is assembled in sim_params in lib.read as np.outer(dz * xi, np.ones((1,v.N))).

    outputs:
    d -- (ndarray, ndim=3) d[q,i,j], gives the qth derivative for each point (i,j)
          for q >= 1, the zeroeth derivative d[0,:,:] = f_old is assigned prior to this function call
          i.e. the assignment is for d[1:, :, :] below, d[1:,:,:].shape = (N-1, x.N, v.N)
    """

    # need to specify the axis in numpy v1.11.0.dev0+fe64f97, has opposite default axis
    # as prior versions!
    Ff = np.fft.fft(f, axis = 0) # or for v-advection if rows are maintained can toggle axis = 1

    D = sim_params['Xi'][z.str] * Ff

    d = np.real(np.fft.ifft(D, axis = 1)) # or for v-advection if rows are maintained can toggle axis = 2 here
    return d

### <font color = "green">DECSKS-v2.0 lib.derivative.trigometric3D_v</font> (new function)

Let's prepare for column-wise differentation (for every row) on an object f with shape (x.N, vx.N).

We require the wave numbes as numerous as the number of physical velocity gridpoints, $N_v$, i.e.

$$\underline{\xi} = (\xi_0, \xi_1, \ldots , \xi_{N_v-1})_{N_v\times 1}$$

Every row is identical, hence we compute the outer product

$$\underline{\underline{\xi}} = \underline{1}_{N_x\times 1} \otimes \underline{\xi}_{N_v\times 1} = \left( \begin{array}{ccc}
\xi_0 & \xi_1 & \xi_{N_v-1} \\
\xi_0 & \xi_1 & \xi_{N_v-1} \\
\vdots & \vdots & \vdots \\
\xi_0 & \xi_1 & \xi_{N_v-1}  \end{array} \right)_{N_x\times N_v}$$

To compute the derivative, we can assemble the object $\underline{\underline{\Xi}}_{N\times N_x\times N_v} = (\Xi_{nm\ell}) = (j\Delta v \xi_{\forall m, \ell})^{(n)}$ for $n\in [0,N), m\in [0, N_x), \ell\in [0, N_v)$, where $n,m, \ell \in \mathbb{N}$. Note, the subscript notation $\forall m, \ell$ means the entries $\Xi_{nm\ell}$ are different for each $\ell$, and all rows $m$ are identical for that given $n$. 

$$\underline{\underline{\Xi}} = (\Xi_{n,m,\ell}) = (j\Delta v\xi_{\forall m,\ell})^{n}\left\{\left( \begin{array}{ccc}
1 & 1 & 1 \\
1 & 1 & 1 \\
\vdots & \vdots & \vdots \\
1 & 1 & 1 \end{array} \right)_{n = 0, N_x\times N_v}
\\ 
\left( \begin{array}{ccc}\
(j\Delta v\xi_0)^1 & (j\Delta v\xi_1)^1 & (j\Delta v\xi_{N_v-1})^1 \\
(j\Delta v\xi_0)^1 & (j\Delta v\xi_1)^1 & (j\Delta v\xi_{N_v-1})^1 \\
\vdots & \vdots & \vdots \\
(j\Delta v\xi_0)^1 & (j\Delta v\xi_1)^1 & (j\Delta v\xi_{N_v-1})^1 \end{array} \right)_{n = 1, N_x\times N_v}
\\ 
\left( \begin{array}{ccc}\
(j\Delta v\xi_0)^2 & (j\Delta v\xi_1)^2 & (j\Delta v\xi_{N_v-1})^2 \\
(j\Delta v\xi_0)^2 & (j\Delta v\xi_1)^2 & (j\Delta v\xi_{N_v-1})^2 \\
\vdots & \vdots & \vdots \\
(j\Delta v\xi_0)^2 & (j\Delta v\xi_1)^2 & (j\Delta v\xi_{N_v-1})^2 \end{array} \right)_{n = 1, N_x\times N_v} \\
\vdots \\
\left( \begin{array}{ccc}\
(j\Delta v\xi_0)^{N-1} & (j\Delta v\xi_1)^{N-1} & (j\Delta v\xi_{N_v-1})^{N-1} \\
(j\Delta v\xi_0)^{N-1} & (j\Delta v\xi_1)^{N-1} & (j\Delta v\xi_{N_v-1})^{N-1} \\
\vdots & \vdots & \vdots \\
(j\Delta v\xi_0)^{N-1} & (j\Delta v\xi_1)^{N-1} & (j\Delta v\xi_{N_v-1})^{N-1} \end{array} \right)_{n = 1, N_x\times N_v}
\right\}
$$

The matrix can be assembled efficiently in numpy:


$$\underline{\underline{\Xi}}_{N\times N_x\times N_v} = (\text{j}\Delta v \underline{\underline{\xi}}_{N_x\times N_v})^{[0, 1, \ldots , N-1]}$$

i.e.

    N_xi = active_dims[1] # for sim_params['Xi']['x'], N_xi = active_dims[0]
    wave_index = np.arange(N_xi)
    xi = np.where(wave_index <= N_xi / 2,
              2*np.pi*wave_index / L,
              2*np.pi*(wave_index - N_xi) / L)

    xi_2D = np.outer(np.ones([1,x.N], xi))
    dn = np.arange(1,N).reshape(N-1, 1, 1)
    Xi = (1j * vx.width * xi) ** dn
    
    

At this point, one option is to find $\mathcal{F}f$ along rows,

    Ff = np.fft.fft(f, axis = 1)
    
here, f.shape = (x.N, vx.N) in our perspective. We then could seek the transformed derivative coefficients $\mathcal{F}d \equiv$<code>D</code> as (suppose as planned, we stoore this matrix Xi in sim_params['Xi']['v']

    D = sim_params['Xi'][z.str[0]]
    
Where now, the stored object D.shape = (N, x.N, vx.N). The inverse is then taken as

    d = np.fft.ifft(D, axis = 2)
    
Note, different axes are toggled each time as compared to the x-advection stepthrough.

### Alternate: transpose the density f

If f = np.transpose(f) so that f.shape = (vx.N, x.N), then we can plan on having sim_params['Xi']['v'].shape = (vx.N, x.N) by transpose operation prior to the start of the simulation as assembled in lib.read.assemble_spectral_derivative_operator, then the same code can be used:

    Ff = np.fft.fft(f, axis = 0)
    
here, f.shape = (x.N, vx.N) in our perspective. We then could seek the transformed derivative coefficients $\mathcal{F}d \equiv$<code>D</code> as (suppose as planned, we stoore this matrix Xi in sim_params['Xi']['v']

    D = sim_params['Xi'][z.str[0]]
    
Where now, the stored object D.shape = (N, x.N, vx.N). The inverse is then taken as

    d = np.fft.ifft(D, axis = 1)
    
Since the axes have been transposed.

In [6]:
import numpy as np

Nx = 2
Nv = 5

A = np.arange(Nv)


np.outer(np.ones(Nx), A)

array([[ 0.,  1.,  2.,  3.,  4.],
       [ 0.,  1.,  2.,  3.,  4.]])

## Computing $[Uf]_{i,j} = \sum_q c_{(i,j)}d_{(i,j)}^{(q)}$

Suppose we have both d and c. How to compute the high order flux [Uf]? 

For x-advection, we have in lib.convect.flux:

    for j in range(vz.N):
        Uf[:,j] = c[:,j].dot(d[:,:,j])
        
There, z = x, and vz = vx.

For v-advection, we have z = vx, and vz = ax. <b>The above still works provided that the shape of Uf is the transpose of the above (x.N, v.N)</b>, so that Uf.shape = (vx.N, ax.N), and ax.N = x.N already. The shape of Uf is initialized just prior in lib.convect.flux as

        Uf = np.zeros(f_old.shape)
        
Thus, if we have f_old.shape in this case to be (vx.N, x.N), then we can reuse all of this. The natural place to do this is at the start of the substep in lib.convect.scheme

    # (0) INITIALIZE FINAL DENSITY CONTAINER AND EXTRACT EVOLVED GRID
    f_final = np.zeros(f_initial.shape)
    f_initial = DECSKS.lib.convect.extract_active_grid(sim_params, f_initial)
    
we can add a conditional check:

    if z.str[0] == 'v': # if the phase space variable being evolved is a physical velocity
        f_initial = np.transpose(f_initial)
        
This isn't an expensive check. Otherwise, we could develop a separate flux function lib.convect.flux_v and lib.convect.flux_x, and toggle an evaluation of the string that corresponds to the function handle flux_handle = "".join(lib.convect.flux_, 'v') or flux_handle = "".join(lib.convect.flux_, 'x'), then call with 

    eval(flux_handle)(*args)
    
then inside that function we explicitly and confidently take

    Uf = np.zeros(f_old.T.shape)
    


<font color = "red">It seems too entangled to take tranposes if z.str = 'v' and happens in too many places. Plan on transposing f, z.prepointmesh, z.postpointmesh, and so on in order to use the same methods</font>, tranpose back after before the return.

## v-advection decision: transpose f early on

Tracing through the code to see how this affects everything....

### lib.convect.scheme

In [ ]:
def scheme(
    f_initial,
    n,
    sim_params,
    z,
    vz
    ):
    """Solves a collection of 1D advection (in z) equations by convected scheme
    and stacks the results in a 2D matrix

    inputs:
    f_initial -- (ndarray, ndim = 2) f[n-1,:,:] if first
             substep in a splitting algorithm or if none
             else, f[n,:,:]
    n -- (int) time step
    sim_params -- (dict) simulation parameters
    z -- (instance) phase space variable
    vz -- (instance) generalized velocity for z


    outputs:
    f_final -- (ndarray, ndim = 1 or 2) f[n,:] or f[n,:,:] updated
               after all steps have been completed
    """

    # (0) INITIALIZE FINAL DENSITY CONTAINER AND EXTRACT EVOLVED GRID
    f_final = np.zeros(f_initial.shape)
    f_initial = DECSKS.lib.convect.extract_active_grid(sim_params, f_initial)

    # (1) PUSH MOVING CELLS an integer number of cells
    z.postpointmesh = advection_step(z)
    # NOTE: this can be done without vz, as all sign information
    # is stored in z.CFL.frac.

    # (*) APPLY BOUNDARY CONDITIONS
    z.postpointmesh = DECSKS.lib.boundaryconditions.periodic(z.postpointmesh, z.N)

    # (2) REMAP DENSITY TO GRID
    f_remapped = remap_step(
                       sim_params,
                       f_initial,
                       n,
                       z,
                       vz
                       )

    # (3) COLLISION STEP (NOT YET IMPLEMENTED)
    # f_new = DECSKS.lib.collisions.collisiontype(f_old, z, n)

    # (4) RETURN FINAL DESTINY (density*)
    f_final = finalize_density(sim_params, f_remapped, f_final)

    return f_final


We can put this inside the function lib.convect.extract_active_grid, but this makes the function handle misleading. Since we require tranpose a few objects, we create a new method for the case if z.str[0] == 'v', lib.domain.velocity_advection

### lib.convect.extract_active_grid

We note in moving forward in lib.convect.scheme that z.postpointmesh is still of dimensions (x.N, vx.N). The next line

    # (1) PUSH MOVING CELLS an integer number of cells
    z.postpointmesh = advection_step(z)


### lib.convect.advection_step

In [ ]:
def advection_step(z):
    """Pushes each z.prepointmesh (index) value by the advection *along each
    column j* (i.e. constant vz.prepointvaluemesh)
    as prescribed by its generalized velocity vz.prepointmeshvalues[:,j].

    This is computed in one calculation by:

        vz.prepointmeshvalues * dt / z.width = CFL.numbers

    the integer part, CFL.int, is pushed here. The residual fraction
    CFL.frac is remapped according to remap_step(*args) in step (2) in
    the orchestrator routine, scheme(*args), above.

    inputs:
    z -- (instance) phase space variable equipped with (among other attributes)

            z.prepointmesh -- (ndarray, ndim=2) initial [i,j] prepoint indices of all MCs

            z.CFL -- (instance) contains CFL numbers, int and frac parts
                z.CFL.numbers -- (ndarray, ndim=2, dtype=float64)
                z.CFL.int -- (ndarray, ndim=2, dtype=int) integer part of CFL numbers
                z.CFL.frac -- (ndarray, ndim=2, dtype=float64) fractional part

    outputs:
    z -- (instance) phase space variable updated with postpointmesh attribute (BCs not applied)

        updated attr:
        z.postpointmesh -- (ndarray, ndim=3, dtype=int), shape = (2, x.N, v.N) matrix containing each pair
                           k[:,i,j] of postpoints for each prepoint [i,j]


    NOTE: Boundary conditions NOT applied at this point

    USAGE NOTE: the map z.postpointmesh[k,i,j] for k = 0 or 1 gives the postpoint
      of the advecting index, it does not give the relevant postpoint duple
      in phase space:

          (z.postpointmesh[k,i,j], j) [for x advection]

      or

          (i, z.postpointmesh[k,i,j]) [for v advection]

      but instead just gives that value z.postpointmesh[k,i,j] of the index
      that changes. Hence, the remap procedure must be encoded with an
      understanding of which index is updated in order to complete the push.

      It then makes sense to speak in terms such as:

          z.postpointmesh[0,i,j] + 1 is a contiguous gridpoint to
                       the index z.postpointmesh[0,i,j] in the positive
                       direction

          z.postpointmesh[0,i,j] - 1 is a contiguous gridpoint to
                       the index z.postpointmesh[0,i,j] in the negative
                       direction

      since we are speaking of one index, not a duple as concerns the postpointmesh
      object
    """
    z.postpointmesh[0,:,:] = z.prepointmesh + z.CFL.int
    z.postpointmesh[1,:,:] = np.sign(z.CFL.frac).astype(int) + z.postpointmesh[0,:,:]

    return z.postpointmesh

All these objects are compatible in dimensions. The attribute z.CFL.int and z.CFL.frac are computed, and so happen to have generalized velocities (for v-advection) that vary in rows, not columns, but this is what is needed here. All the points are updated accordingly. We can return a tranposed object if needed. We also require the tranpose of z.CFL.frac for the computation of the correctors c. <b>It might be more natural to create the velocity variables wiht the tranposed dimensions rather than initialize (x.N, vx.N) and transpose as needed. </b>

initializing:

    vx.CFL.numbers.shape = (vx.N, x.N)
    
then automatically creates the objects:

    vx.CFL.int
    vx.CFL.frac
    
to be of the same shape. 

<b><font color = "red">(TODO) We may consider this later</font></b>, for now we implement an obvious stepthrough by forming the same objects and tranposing as needed.

We add the following function call in lib.convect.scheme on lines L30-31:

In [ ]:
def scheme(
    f_initial,
    n,
    sim_params,
    z,
    vz
    ):
    """Solves a collection of 1D advection (in z) equations by convected scheme
    and stacks the results in a 2D matrix

    inputs:
    f_initial -- (ndarray, ndim = 2) f[n-1,:,:] if first
             substep in a splitting algorithm or if none
             else, f[n,:,:]
    n -- (int) time step
    sim_params -- (dict) simulation parameters
    z -- (instance) phase space variable
    vz -- (instance) generalized velocity for z


    outputs:
    f_final -- (ndarray, ndim = 1 or 2) f[n,:] or f[n,:,:] updated
               after all steps have been completed
    """

    # (0) INITIALIZE FINAL DENSITY CONTAINER AND EXTRACT EVOLVED GRID
    f_final = np.zeros(f_initial.shape)
    f_initial = extract_active_grid(f_initial, z, sim_params)

    if z.str[0] == 'v':
        f_initial, z = lib.domain.velocity_advection_prep(f_initial, z)

    # (1) PUSH MOVING CELLS an integer number of cells
    z.postpointmesh = advection_step(z)

    # (*) APPLY BOUNDARY CONDITIONS
    z.postpointmesh = DECSKS.lib.boundaryconditions.periodic(z.postpointmesh, z.N)

    # (2) REMAP DENSITY TO GRID
    f_remapped = remap_step(
                       sim_params,
                       f_initial,
                       n,
                       z,
                       vz
                       )

    # (3) COLLISION STEP (NOT YET IMPLEMENTED)
    # f_new = DECSKS.lib.collisions.collisiontype(f_old, z, n)

    # (4) RETURN FINAL DESTINY (density*)
    f_final = finalize_density(sim_params, f_remapped, f_final)

    return f_final

and the associated function:

### lib.domain.velocity_advection_prep

In [ ]:
def velocity_advection_prep(f_initial, z):
    """
    When advecting physical velocity variables, the implementation
    requires several transpositions. This method performs those
    operations

    inputs:
    f_initial -- (ndarray, ndim=2) shape = (x.N, vx.N)
    z -- (instance) phase space variable being evolved

        z.prepointmesh -- (ndarray, ndim=2)
        z.postpointmesh -- (ndarray, ndim=3), shape = (2, x.N, vx.N)

        z.CFL.frac -- (ndarray, ndim=2) shape = (x.N, vx.N)
        z.CFL.int -- (ndarray, ndim=2) shape = (x.N, vx.N)

    outputs:

    f_initial -- (ndarray, ndim=2) shape = (vx.N, x.N)
    z -- (instance) phase space variable being evolved

        z.prepointmesh -- (ndarray, ndim=2) shape = (vx.N, x.N)
        z.postpointmesh -- (ndarray, ndim=3), shape = (2, vx.N, x.N)

        z.CFL.frac -- (ndarray, ndim=2) shape = (vx.N, x.N)
        z.CFL.int -- (ndarray, ndim=2) shape = (vx.N, x.N)


    reverting to the appropriate dimensions for f_initial --> f_final
    is done at the final step of lib.convect in the function call
    lib.convect.finalize_density (also, where periodic BCs are applied
    if specified)
    """

    f_initial = np.transpose(f_initial)

    z.prepointmesh = np.transpose(z.postpointmesh)
    z.postpointmesh = np.transpose(z.postpointmesh, (0,2,1))

    z.CFL.frac = np.transpose(z.CFL.frac)
    z.CFL.int = np.transpose(z.CFL.int)

    return f_initial, z

Everything is handled properly, the next line to be concerned about is

In [ ]:
    # (2) REMAP DENSITY TO GRID
    f_remapped = remap_step(
                       sim_params,
                       f_initial,
                       n,
                       z,
                       vz
                       )

which steps into lib.convect.flux:

In [ ]:
    Uf = flux(
        sim_params,
        f_old,
        z, vz
        )

Calling all correctors! Calling all correctors!

In [ ]:
## excerpt from lib.convect.flux

    c = DECSKS.lib.HOC.correctors(sim_params, z, vz)

    # evaluate derivatives q = 0, 1, 2, ... N-1 (zeroeth is density itself)

    # calls lib.derivatives.fd or lib.derivatives.fourier based on the
    # HOC specified in etc/params.dat, sim_params['derivative_method']
    # contains the related function handle as a string
    d = eval(sim_params['derivative_method'])(f_old, z, vz, sim_params)

    # compute high order fluxes column-by-column
    Uf = np.zeros(f_old.shape)
    for j in range(vz.N):
        Uf[:,j] = c[:,j].dot(d[:,:,j])

In [ ]:
## excerpt from lib.HOC.correctors

    Beta_func_handle = "%s%s" % ("Beta_matrix_", z.str[0])

    B = eval(Beta_func_handle)(sim_params, z, vz)
    c = sim_params['I_alternating'].dot(B)
    return c

It looks like we do not require two Beta_matrix versions for x and v given our previous transpositions.

In [17]:
def Beta_matrix(sim_params, z, vz):
    """constructs the B matrix, whose columns are
    the beta vectors (shape = N x 1) for each value
    of the generalized velocity for the advecting
    variable z.

    See DECSKS-09 part 1 for details of this calculation

    inputs:
    sim_params -- (dict) simulation parameters
    z.CFL.frac -- (ndarray, ndim=2) contains the fractional CFL numbers
                  for every [i,j]
    """

    # local copies of A matrices
    A_neg = sim_params['A_matrix']['-1']
    A_pos = sim_params['A_matrix']['1']

    N_arr = np.outer(np.arange(1,sim_params['N']+1), np.ones([1,vz.N]))

    # for broadcasting operations below to an N x vz.N array
    # we require an identically dimensioned matrix. Hence, we slice the rows up to N values.
    # There is no loss of information here given that every row entry in the z.CFL.frac matrix
    # is constant, given z.CFL.frac = z.CFL.frac (vz.prepointvaluemesh)

    # Naming per DECSKS-09 notebook:
    #
    #        alpha = z.CFL.frac, shape: (z.N, vz.N)
    #        alpha_hat = truncatedilde[q,j] = alpha_hat ** q,q = 0, 1, ... N-1
    #

    alpha_hat = z.CFL.frac[:sim_params['N'],:z.CFL.frac.shape[1]]
    alpha_tilde = ma.array(alpha_hat ** N_arr / scipy.misc.factorial(N_arr))

    mask_neg = (alpha_hat < 0)
    mask_pos = np.logical_not(mask_neg)

    # mask out negative values, leave only positives (>= 0)
    alpha_tilde.mask = mask_neg

    # operate on only positive vlaues (>= 0)
    beta_pos = ma.dot(A_pos, alpha_tilde)

    # mask out positive (>= 0), leave only negatives
    alpha_tilde.mask = mask_pos

    # operate on only negative values
    beta_neg = ma.dot(A_neg, alpha_tilde)

    # consolidate all columns in a single matrix
    B = np.zeros([sim_params['N'], sim_params['N' + vz.str]])

    # wherever beta_neg.mask is False (i.e. unmasked value), assign beta_neg, otherwise beta_pos
    B = np.where(mask_neg == True, beta_neg.data, beta_pos.data)

    return B

this works up until the second before last line:

    # consolidate all columns in a single matrix
    B = np.zeros([sim_params['N'], sim_params['N' + vz.str]])

This was an unnecessary implementation anyway, we could have toggled the second dimension limit by access vz.N. This is like the above because it was from a time when the velocity vz was not being passed around like it is now so we didn't have direct access and instead appealed to the dictinoary sim_params to consttruct the proper dimensions. So, we can change the above

In [ ]:
## bottom of lib.HOC.Beta_matrix

    # consolidate all columns in a single matrix
    B = np.zeros([sim_params['N'], vz.N])

    # wherever beta_neg.mask is False (i.e. unmasked value), assign beta_neg, otherwise beta_pos
    B = np.where(mask_neg == True, beta_neg.data, beta_pos.data)

    return B

Then B.shape = (N, x.N) as needed for the v-advection case, and we return to the parent function to compute:

    c = sim_params['I_alternating'].dot(B)
    
Where I_alternating.shape = (N,N), and gives c with c.shape = (N, x.N) as needed. <b>Since the same Beta_matrix function can be used for both x and v advection, we remove the generic function call eval(Beta_function_handle) above, and instead just call the function directly in lib.HOC.correctors


In [ ]:
    B = Beta_matrix(sim_params, z, vz)
    c = sim_params['I_alternating'].dot(B)
    return c

We then return to lib.convect.flux to compute

    # calls lib.derivatives.fd or lib.derivatives.fourier based on the
    # HOC specified in etc/params.dat, sim_params['derivative_method']
    # contains the related function handle as a string
    d = eval(sim_params['derivative_method'])(f_old, z, vz, sim_params)
    
Let's take the case of the function handle being lib.derivatives.fourier

In [ ]:
def fourier(f_old, z, vz, sim_params):
    """Orchestrates the computation of sim_params['N'] - 1 fourier
    derivatives and stores these in a matrix of derivative
    coefficients d. The zeroeth entry contains the zereoth
    derivative, i.e. the density itself

    inputs:
    f_old -- (ndarray, ndim=2) density from previous time substep
    z -- (instance) phase space variable being advected, here
         we are taking d/dz

    vz -- (instance) generalized velocity for z
    sim_params -- (dict) simulation parameters dictionary

    outputs:
    d -- (ndarray, ndim=3), shape = (N, x.N, v.N), the
         entry d[q,i,j] gives the qth derivative coefficient
         at a point [i,j]
    """
    d = np.zeros([sim_params['N'], z.N, vz.N])

    # zeroeth derivative coefficient is the density itself
    d[0,:,:] = f_old

    # if corrections indicated in etc/params.dat, need higher order terms
    if sim_params['N'] > 1:
        d[1:,:,:] = trigonometric3D(f_old, z, sim_params)

    return d


This initializes d

    d = np.zeros([sim_params['N'], z.N, vz.N])

with d.shape = (N, vx.N, x.N), also consistent with our formulation thus far. Then, we take

    d[0,:,:] = f_old
    
since f_old.shape = (vx.N, x.N) by transposition from the outset, the dimensions are aligned accordingly.

if corrections are desired, then the derivatives need to be computed, calling lib.derivatives.trigonometric3D:


In [ ]:
def trigonometric3D(f, z, sim_params):
    """Computes derivatives of density for derivative coeffs d in FN methods

    inputs:
    f -- (ndarray, ndim=2) density at a given time step
    sim_params['Xi']['x' or 'vx'] -- (ndarray, ndim=3) Xi = (1j*dz*xi) ** [[[1,2,3,...,N-1]]], where xi.shape (x.N, v.N)
          is assembled in sim_params in lib.read as np.outer(dz * xi, np.ones((1,v.N))).

    outputs:
    d -- (ndarray, ndim=3) d[q,i,j], gives the qth derivative for each point (i,j)
          for q >= 1, the zeroeth derivative d[0,:,:] = f_old is assigned prior to this function call
          i.e. the assignment is for d[1:, :, :] below, d[1:,:,:].shape = (N-1, x.N, v.N)
    """

    # need to specify the axis in numpy v1.11.0.dev0+fe64f97, has opposite default axis
    # as prior versions!
    Ff = np.fft.fft(f, axis = 0) # or for v-advection if rows are maintained can toggle axis = 1

    D = sim_params['Xi'][z.str] * Ff    # selects the Xi matrix based on 'x' or 'vx'

    d = np.real(np.fft.ifft(D, axis = 1)) # or for v-advection if rows are maintained can toggle axis = 2 here
    return d

We transform f along axis = 0, i.e. rows. In v-advection, the acceleration varies with row formerly, but upon transposition they vary with column. Thus, each column is a 1D advection problem in vz = ax, hence the direction of differentiation is along the rows, which is appropriately chosen above in the computation of 

    Ff = np.fft.fft(f, axis = 0)
    
then we multiply by the relevant Xi matrix according to vx or x. The next line needs to compute the numpy point-wise multiplication 

    D = sim_params['Xi']['vx'] * Ff
    
Thus, these two objects need to be the same shape. <font color = "red">this is a change from above</font> we note we must modify the provisional construction for Xi by returning the transpose to the key/value sim_params['Xi']['vx'] so that its shape is (vx.N, x.N) as needed, since Ff.shape = (vx.N, x.N), and f.shape = (vx.N, x.N)



In [ ]:
## lib.read.assemble_spectral_derivative_operator excerpt (bottom)

    # for v advection variables, we construct the case for 1D1V:

    N_xi, Nrows = active_dims[1], active_dims[0]

    wave_index = np.arange(N_xi)
    xi = np.where(wave_index <= N_xi / 2,
              2*np.pi*wave_index / Lvx,
              2*np.pi*(wave_index - N_xi) / Lvx)

    xi_2D = np.outer(np.ones(N_rows), xi) # wavenumbers enumerated by row, copies in Nv columns

    xi_2D = np.transpose (xi_2D) # dimensions are (vx.N, x.N) now to match v-advection cases
    dn = np.arange(1,N).reshape(N-1,1,1)
    Xi['vx'] = (1j * width * xi_2D) ** dn

    return Xi, xi

We also note that we need to manage two wave number vectors xi, we can stuff these into their own dictionary just like we did for Xi. The whole method then takes the form:

### lib.read.assemble_spectral_derivative_operator

In [ ]:
def assemble_spectral_derivative_operator(total_dims, active_dims,
                                          ax, bx, avx, bvx,
                                          N):
    """Returns a dictionary Xi with key/value pairs:

        Xi['x'] -- (ndarray, ndim=3, dtype=complex)
        Xi['vx'] -- (ndarray, ndim=3, dtype=complex)

    Each of these matrices correspond to a matrix with entries

      $$Xi = ((Xi)_{q,i,j}) = 1j * (\Delta z \xi_{i,j})^q$$

    USAGE NOTE: computing Xi * Ff, where Ff is numpy.fft.fft(f)
    and f.shape = (x.N, vx.N) produces the Fourier transform
    of the derivative coefficients $F[d] \equiv D$, where
    D[q,i,j] corresponds to the qth order derivative coefficient
    at a phase space location [i,j]. The method
    lib.derivatives.trigonometric3D takes the row-wise inverse
    transform so that the tensor d[q,i,j] is generated.
    """

    Xi = {}
    xi = {}

    Lx = float(bx - ax)
    xwidth = float((bx - ax) / (total_dims[0] - 1))

    Lvx = float(bvx - avx)
    vxwidth = float((bvx - avx) / (total_dims[1] - 1))

    # TODO x-differentiation, haven't figured out what v-differentaition looks like yet
    N_xi, N_cols = active_dims[0], active_dims[1]

    # for v-differentiation, number of wavenumbers N_xi = active_dims[1], N_rows (or
    # N_cols if we do transpose operations) = active_dims[0]

    wave_index = np.arange(N_xi)
    xi = np.where(wave_index <= N_xi / 2,
              2*np.pi*wave_index / Lx,
              2*np.pi*(wave_index - N_xi) / Lx)

    xi['x'] = xi
    xi_2D = np.outer(xi, np.ones(N_cols)) # wavenumbers enumerated by row, copies in Nv columns

    dn = np.arange(1,N).reshape(N-1,1,1)
    Xi['x'] = (1j * width * xi_2D) ** dn

    # if other configuration variables involved, i.e. 'y', 'z', will need to develop these as well

    # for v advection variables, we construct the case for 1D1V:

    N_xi, Nrows = active_dims[1], active_dims[0]

    wave_index = np.arange(N_xi)
    xi = np.where(wave_index <= N_xi / 2,
              2*np.pi*wave_index / Lvx,
              2*np.pi*(wave_index - N_xi) / Lvx)

    xi['vx'] = xi
    xi_2D = np.outer(np.ones(N_rows), xi) # wavenumbers enumerated by row, copies in Nv columns

    xi_2D = np.transpose (xi_2D) # dimensions are (vx.N, x.N) now to match v-advection cases
    dn = np.arange(1,N).reshape(N-1,1,1)
    Xi['vx'] = (1j * width * xi_2D) ** dn

    return Xi, xi

Returning then to lib.derivatives.trigonometric3D, we compute

    D = sim_params['Xi'][z.str] * Ff    # selects the Xi matrix based on 'x' or 'vx'

Without issue with the aforementioned changes.

The final two lines

    d = np.real(np.fft.ifft(D, axis = 1)) 
    return d

This is computed along the rows of each (i.e. the matrix is three dimensional), which is the direction of the transpose that is needed.

We then step back into lib.convect.flux after a long trip to compute:

    # compute high order fluxes column-by-column
    Uf = np.zeros(f_old.shape)
    for j in range(vz.N):
        Uf[:,j] = c[:,j].dot(d[:,:,j])
        
 Uf is initialized with shape Uf.shape = (vx.N, x.N) as needed, then we loop through columns and perform the required dot products. This is consistent.
 
The function is thrown to be caught by the receiver of the limiter:

    # enforce flux limiter to ensure positivity and restrict numerical overflow
    Uf = flux_limiter(f_old, Uf, z)

    return Uf
    
All good!



### still to verify works with lib.derivatives.fd

Stepping back into lib.convect.remap_step, we call the remap_assignment method:

    # remap to nearest neighbor cell center
    f_k1 = remap_assignment(
        f_old,
        Uf,
        z.postpointmesh[0,:,:],
        z,
        vz,
        index = 'nearest'    # remaps to nearest neighbor index
        )

    # remap to contiguous cell center
    f_k2 = remap_assignment(
        f_old,
        Uf,
        z.postpointmesh[1,:,:],
        z,
        vz,
        index = 'contiguous'    # remaps to contiguous neighbor of above
        )

    f_remapped = f_k1 + f_k2
    
Note, <b>vz is passed through a lot of functions here and prior to this point. It should be noted that as far as the advection is concerned, the values on vz have already been used. We pass around vz in these routines only to access the dimension vz.N, below we see we actually have to access vz.prepointmesh in order to slice appropriately. <font color = "red">We should go back to lib.split and transpose the acceleration or otherwise change the instantiation in lib.domain.Setup ----- Actually, we just append this line to the velocity_advection_prep method</b>:

    vz.prepointmesh = np.transpose(vz.prepointmesh)

And, that is that. This requires including it in the argument and function return. This is done as well (not shown here as the changes are clear).

In [ ]:
def remap_assignment(
        f_old,
        Uf,
        zpostpointmesh,
        z,
        vz,
        index = 'nearest'
        ):
    """Remaps the z.N MCs to Eulerian mesh at indices k1 and k2

    inputs:
    f_old -- (ndarray, dim=1) density from previous time step
    Uf -- (ndarray, dim=1) the normalized flux used in the CS update
    z -- (instance) phase space variable whose MCs are convected
    n -- (int) current time step

    outputs:
    f_new -- (ndarray, dim=1) density with f_old remapped to
             according to the mapping

            [z.prepointmesh, vz.prepointmesh] --> [k, vz.prepointmesh]

    """
    mask_neg =  (z.CFL.frac < 0)
    mask_pos = np.logical_not(mask_neg)

    f_old_ma = ma.array(f_old)
    f_pos, f_neg = ma.zeros(f_old.shape), ma.zeros(f_old.shape)

    Uf_ma = ma.array(Uf)

    if index == 'nearest':
        # mask out negative values
        f_old_ma.mask = mask_neg
        Uf_ma.mask = mask_neg

        f_pos[ zpostpointmesh, vz.prepointmesh ] = f_old_ma - Uf_ma

        # mask out all positive values
        f_old_ma.mask = mask_pos
        Uf_ma.mask = mask_pos

        f_neg[ zpostpointmesh, vz.prepointmesh ] = f_old_ma + Uf_ma

    elif index == 'contiguous':
        # mask out negative values
        f_old_ma.mask = mask_neg
        Uf_ma.mask = mask_neg

        f_pos[ zpostpointmesh, vz.prepointmesh ] = Uf_ma

        # mask out all positive values
        f_old_ma.mask = mask_pos
        Uf_ma.mask = mask_pos

        f_neg[ zpostpointmesh, vz.prepointmesh ] = -Uf_ma

    # "wherever there is negative data, assign f_neg, else assign f_pos
    f_new = np.where(mask_neg == True, f_neg.data, f_pos.data)

    return f_new


With the required transposition of the prepointmesh values for the consistent index slicing alongside z.prepointmesh (shape = (vx.N, x.N)

Returning to lib.convect.remap_step, we then step into:


    f_remapped = f_k1 + f_k2
    
    # global check on density conservation
    DECSKS.lib.density.global_conservation_check(sim_params, f_remapped, n)

    return f_remapped

of which peeking in to the function reveals consistent object handling. This is returned to the object f_remapped in the orchestrator routine lib.convect.scheme.

Finally, inside lib.convect.scheme we finalize the density:

    # (4) RETURN FINAL DESTINY (density*)
    f_final = finalize_density(sim_params, f_remapped, f_final)

We add an if clause to catch v-advection cases and transpose them back to the appropriate dimensinos (x.N, vx.N).

In [ ]:
def finalize_density(sim_params, f_remapped, f_final):
    """
    returns a final density. For all but PBCs, f_new = f_final since
    our setup is such that (z1.N, z2.N) moving cells are evolved.

    The bookkeeping is such that, e.g. in 1D

          non-periodic BCs : z.N = z.Ngridpoints

          periodic BCs     : z.N = z.Ngridpoints - 1
                             f[z.Ngridpoints-1] = f[0] by periodicity

    We use the associated generalization to two dimensions, e.g.

          non-periodic BCs: z1.N = z1.Ngridpoints
                            z2.N = z2.Ngridpoints

                            hence f_final = f_new

          periodic BCs     : z.N = z.Ngridpoints - 1
                             f_final[z1.Ngridpoints - 1, :] = f_new[0,:]
                             f_final[:, z2.Ngridpoints - 1] = f_new[:,0]
  """
    # TODO currently assuming that both dimensions are periodic, need to generalize
    # this in the future

    # assign all active grid points to grid values on f_final
    f_final[:f_remapped.shape[0], :f_remapped.shape[1]] = f_remapped

    f_final[f_remapped.shape[0]+1,:] = f_remapped[0,:]
    f_final[:, f_remapped.shape[1] + 1] = f_remapped[:,0]

    # TODO consider if you have to do this

    if z.str[0] == 'v':
        f_final = np.transpose(f_final)

    return f_final

OK! Lastly, check out lib.derivatives.fd

### lib.derivatives.fd

In [ ]:
def fd(f, z, vz, sim_params):
    """computes the derivative coefficient tensor d (shape = (N, z.N, z.N)).

    Note, the below is difficult to read but has been adopted to not needlessly store local
    copies of large arrays. The dictionary Wz = sim_params['W'][z.str] (shape = (N, x.N, v.N))
    contains each Wz[dn,:,:] 2D array of difference coefficients for the dn-th derivative.

    The np.dot product can lumber signnificantly for large enough arrays (100x+ slower) if no care is taken.
    Here, we bypass the internal looping (hence, reshaping) mechanics by manually reshaping it
    once and performing the dot product by falling back on the 2D GEMM routine.

    this implementation was chosen based on consult of the stackoverflow community:

        http://stackoverflow.com/questions/33004551/
            why-is-b-numpy-dota-x-so-much-slower-looping-through-doing-bi-numpy

    inputs:
    f -- (ndarray, ndim=2) density from previous time substep, 
         shape = (x.N, vx.N) if advecting in configuration
         shape = (vx.N, x.N) if advecting in velocity
         
    z -- (instance) phase space variable being advected, here
         we are taking d/dz

    vz -- (instance) generalized velocity for z
    sim_params -- (dict) simulation parameters dictionary

    outputs:
    d -- (ndarray, ndim=3), shape = (N, z.N, vz.N), the
         entry d[q,i,j] gives the qth derivative coefficient
         at a point [i,j]

         for x-advection, the ordering is [q, x[i,j], vx[i,j]]
         for v-advection, the ordering is [q, vx[i,j], x[i,j]] (transpose in axes (0,2,1))

    """
    d = np.zeros([sim_params['N'], z.N, vz.N])
    dim1, dim2, dim3 = d.shape
    # zeroeth derivative coefficient is the density itself
    d[0,:,:] = f

    # if corrections indicated in etc/params.dat, need higher order terms
    if sim_params['N'] > 1:
        d[1:,:,:] = np.dot( sim_params['W'][z.str][1,:,:].reshape(-1,sim_params['W'][z.str].shape[-1]), f).reshape(
            sim_params['W'][z.str].shape[0] - 1, # we are computing derivatives 1, 2, ... , first index 0 not included
            sim_params['W'][z.str].shape[1],
            sim_params['W'][z.str].shape[2])

    return d

This all handled accordingly, the differentiation occurs over the rows for each column where rows span the vx[:,j] dimension which all i are characterized by common ax[i,j] since the transpose has been taken 

#### Finalizing changes to allow for totally FD implementation

What remains is the small modifications needed in the lib.split routine where accelerations are calculated. In finite difference formulations, we developed a 6th order Poisson solver for phi (not Gauss solver for E). These small changes need to be traced through as far as they go to see that everything is consistent. We label here a scheme lib.split_fd for clarity, but may decide to change the label later. We can also easily call this routine once at the start of simulation without explicit reference, but by doing something like

    eval(split_function_handle)(*args)
    
where 

    split_function_handle = '_'.join(('DECSKS.lib.split', HOC.lower()))
    split_function_handle = '.'.join((split_function_handle, 'scheme'))

i.e.

    split_function_handle = "split_fd"

or

    split_function_handle = "split_fourier"
    
Hence, we relabel lib.split to these two names in order to call DECSKS.lib.split_fd.scheme or DECSKS.lib.split_fourier.scheme according to the input file specifications.